<a href="https://colab.research.google.com/github/pratap977/CS6910/blob/main/cnn_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install split-folders

In [2]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 7.7 MB/s 
     |████████████████████████████████| 144 kB 59.9 MB/s 
     |████████████████████████████████| 181 kB 53.5 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=2f315320ab3ecabc60e406a2e899ba2a794ac889e04494b0ba76e92982c1cf08
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from PIL import Image as im
import math as mh
from matplotlib import pyplot
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Activation,Dense,Flatten
from keras.preprocessing.image import load_img as im
from keras.preprocessing.image import save_img as sim
import os
from random import shuffle
import numpy as np
import random
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers,models
import splitfolders
import keras
from PIL import Image
from torchvision import transforms
from tensorflow.keras.applications import Xception,InceptionV3,InceptionResNetV2,ResNet50
import keras
import wandb
wandb.login()
# keras.backend.set_image_data_format('channels_first')

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
#gettig labels for different classes and assigning values
class_labels = os.listdir("/content/drive/MyDrive/outputfin/train")
lab={l:i for l,i in zip(class_labels,range(10))}




##############################################################
#function for prepating data takes two parameters
#1.drive_path - path to drive i.e training or validataion or test
#2.want_aug - Takes 'YES' or 'NO' values to gather info wheter to augment data or not
def prepare_data(drive_path,want_aug):
  imgs_tr = []
  truth_tr = []
  #data augmentation rules here
  data_augmentation  = transforms.Compose([
          transforms.CenterCrop(224),
          transforms.ToTensor(),
          transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
      ])
  for l in class_labels:
    #### for each folder in the val dir we iterate though all the files 
    if l.startswith("."):
      continue
    imgs_per_class = os.listdir(os.path.join(drive_path, l)) #getting list of file names
    for img in imgs_per_class:
      #iterating to each file in the folder 
      img_path = os.path.join(drive_path, l, img)
      img=Image.open(img_path)
      img = img.resize((224,224))
      if img.mode == 'L':
                  continue
      if(want_aug=='YES'):
        normalized_image = data_augmentation(img)
        imgs_tr.append(np.asarray(normalized_image).transpose(1,2,0))
      else:
        normalized_image = img
        imgs_tr.append(np.asarray(normalized_image))
      truth_tr.append(lab[l])
      
  #contain augmented images(imgs_tr) and labels(truth_tr)
  return np.asarray(imgs_tr) , np.asarray(truth_tr) 



#####################################################################
# Preparing training set (with augmentation), validation set and test test
####################################################################
# Preparing training data with augmentation
# Loading data from directory


#############augmented Training data############ 
x_train,y_train=prepare_data("/content/drive/MyDrive/outputfin/train",'YES')


#############augmented validation data############ 
x_val,y_val=prepare_data("/content/drive/MyDrive/outputfin/val",'YES')

#############augmented test data############
x_test,y_test=prepare_data("/content/drive/MyDrive/inaturalist_12K/val",'YES')




#####################################################################
# Preparing training set (without augmentation), validation set and test test
####################################################################
# Preparing training data withou augmentation
# Loading data from directory
#############augmented Training data############ 
x_train_un,y_train_un=prepare_data("/content/drive/MyDrive/outputfin/train",'NO')


#############augmented validation data############ 
x_val_un,y_val_un=prepare_data("/content/drive/MyDrive/outputfin/val",'NO')

#############augmented test data############
x_test_un,y_test_un=prepare_data("/content/drive/MyDrive/inaturalist_12K/val",'NO')




In [ ]:
xception=tf.keras.applications.Xception(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

In [ ]:
type(xception)

tensorflow.python.keras.engine.functional.Functional

In [ ]:
def Tranfer_Learn(type):
  model=keras.models.Sequential()
  if(type=='Xception'):
    model.add(Xception(weights='imagenet',input_shape=(224,224,3),include_top=False))
  elif(type=='InceptionV3'):
    model.add(InceptionV3(weights='imagenet',input_shape=(224,224,3),include_top=False))
  elif(type=='ResNet50'):
    model.add(ResNet50(weights='imagenet',input_shape=(224,224,3),include_top=False))
  elif(type=='InceptionResNetV2'):
    model.add(InceptionResNetV2(weights='imagenet',input_shape=(224,224,3),include_top=False))
    
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(10,activation='softmax'))
  # model.summary()
  model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
  model.fit(x_train, y_train,epochs=1, validation_split=0.1)
  loss,acc = model.evaluate(x_test, y_test)
  print(loss,acc)
  #wandb.log({"loss":loss,"acc":acc})

In [ ]:
Tranfer_Learn('InceptionV3')

87924736/87910968 [==============================] - 1s 0us/step


AttributeError: ignored

In [ ]:
model=keras.models.Sequential()
model.add(Xception(weights='imagenet',input_shape=(299,299,3),include_top=True))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10,activation='softmax'))
model.summary()

In [ ]:
b=Xception(weights='imagenet',input_shape=(299,299,3),include_top=True)
len(b.layers)

91897856/91884032 [==============================] - 2s 0us/step


134

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
model.predict(x_train[0])

In [ ]:
model.fit(x_train, y_train,batchsize=20,epochs=1, validation_split=0.1)

10/23 [============>.................] - ETA: 10:37 - loss: 2.2918 - accuracy: 0.1972

KeyboardInterrupt: ignored

In [6]:
def finetune(type,freez,epoch,optim,batch,neurons,activ,augmentation):
  model=keras.models.Sequential()
  
  if(type=='Xception'):
    base=Xception(weights='imagenet',input_shape=(224,224,3),include_top=False)
  elif(type=='InceptionV3'):
    base=InceptionV3(weights='imagenet',input_shape=(224,224,3),include_top=False)
  elif(type=='ResNet50'):
    base=ResNet50(weights='imagenet',input_shape=(224,224,3),include_top=False)
  elif(type=='InceptionResNetV2'):
    base=InceptionResNetV2(weights='imagenet',input_shape=(224,224,3),include_top=False)

  if(freez==-1):
    base.trainable=False
  elif(freez<0):
    for i in range(max(-len(base.layers),freez+1),0,1):
      base.layers[i].trainable = False
  elif(freez>0):
    for i in range(0,min(len(base.layers),freez),1):
      base.layers[i].trainable = False 


  model.add(base)  
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(neurons,activation=activ))
  # model.add(keras.layers.Dense(neurons,activation=activ))
  model.add(keras.layers.Dense(10,activation='softmax'))

  
  model.compile(loss="sparse_categorical_crossentropy", optimizer=optim, metrics=["accuracy"])
  if(augmentation=='YES'):
    model.fit(x_train, y_train, batch_size=batch,epochs=epoch,validation_data=(x_val,y_val),callbacks=[tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', patience = 5),
                                            wandb.keras.WandbCallback()])
    # model.fit(x_train, y_train, batch_size=batch,epochs=epoch,validation_data=(x_val,y_val))
    tloss,tacc=model.evaluate(x_val,y_val)
    return tloss,tacc
  else:
    model.fit(x_train_un, y_train_un, batch_size=batch,epochs=epoch,validation_data=(x_val_un,y_val_un),callbacks=[tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', patience = 5),
                                            wandb.keras.WandbCallback()])
    tloss,tacc=model.evaluate(x_val_un,y_val_un)
    return tloss,tacc
  

In [ ]:
freez=-1

In [12]:
finetune('ResNet50',-1,1,tf.keras.optimizers.SGD(0.01),64,'YES')

32/32 [==============================] - 2s 60ms/step - loss: 158.1784 - accuracy: 0.1622


In [12]:
sweep_configer = {
                  'method': 'random',
                  'metric':{
                            'name':'val_accuracy',
                            'goal':'maximize'
                            },
                  'parameters': {
                                'mod'   : {'values': [ 'InceptionResNetV2']},
                                'freez'   :{'values':[-1,-2]},
                                'epochs'    :{'values':[10,8,7]},  
                                 'optimizer'               : {'values' :['adam','rmsprop'] },
                                  'lr'                      : {'values' :[1e-3,1e-4,3e-3,3e-4] },
                                 'neurons' :{'values':[128,512,64,32,16]},
                                 'batchsize'               : {'values' :[16,32,64,128] },
                                 'activ':{'values':['sigmoid','relu']},
                                 'augmentation' : {'values':['YES','NO']},                       
                              }
                }

In [8]:
def hypertune():
  wandb.init(config=sweep_configer,magic=True)
  if(wandb.config.optimizer=="sgd"):
    optim = tf.keras.optimizers.SGD(learning_rate=wandb.config.lr)
  elif(wandb.config.optimizer=="adam"):
    optim = tf.keras.optimizers.Adam(learning_rate=wandb.config.lr)
  elif(wandb.config.optimizer=="rmsprop"):
    optim = tf.keras.optimizers.RMSprop(learning_rate=wandb.config.lr)
  tloss,tacc=finetune(wandb.config.mod,wandb.config.freez,wandb.config.epochs,optim,wandb.config.batchsize,wandb.config.neurons,wandb.config.activ,wandb.config.augmentation)
  wandb.log({"Test loss":tloss,"Test Accuracy":tacc})



In [ ]:


sweep_id = wandb.sweep(sweep_configer,project='A2-Part-BB')
wandb.agent(sweep_id, function=hypertune,project='A2-Part-BB')

Create sweep with ID: yri6tvno
Sweep URL: https://wandb.ai/pratap49/A2-Part-BB/sweeps/yri6tvno


wandb: Agent Starting Run: b26zb65t with config:
wandb: 	activ: relu
wandb: 	augmentation: YES
wandb: 	batchsize: 64
wandb: 	epochs: 8
wandb: 	freez: -1
wandb: 	lr: 0.0001
wandb: 	mod: InceptionResNetV2
wandb: 	neurons: 16
wandb: 	optimizer: rmsprop


Run b26zb65t errored: ResourceExhaustedError()
wandb: ERROR Run b26zb65t errored: ResourceExhaustedError()
wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: yt6desbn with config:
wandb: 	activ: relu
wandb: 	augmentation: YES
wandb: 	batchsize: 32
wandb: 	epochs: 7
wandb: 	freez: -1
wandb: 	lr: 0.0003
wandb: 	mod: InceptionResNetV2
wandb: 	neurons: 16
wandb: 	optimizer: adam


In [ ]:
model.trainable =False

In [ ]:
%cd /content

/content


Part-3

In [ ]:
!git clone https://github.com/pjreddie/darknet
%cd darknet
!make

In [ ]:
!wget https://pjreddie.com/media/files/yolov2.weights

--2021-04-14 17:42:40--  https://pjreddie.com/media/files/yolov2.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203934260 (194M) [application/octet-stream]
Saving to: ‘yolov2.weights’

yolov2.weights      100%[===================>] 194.49M  94.4MB/s    in 2.1s    

2021-04-14 17:42:42 (94.4 MB/s) - ‘yolov2.weights’ saved [203934260/203934260]



In [ ]:
!./darknet detect demo cfg/coco.data cfg/yolov3.cfg yolov3.weights -dont_show testForYolo.mp4 -i 0 -out_filename yolov3output.avi

In [ ]:
 !./darknet detect cfg/yolov2.cfg yolov2.weights /content/darknet/data/yoltest.mp4

layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   608 x 608 x   3   ->   608 x 608 x  32  0.639 BFLOPs
    1 max          2 x 2 / 2   608 x 608 x  32   ->   304 x 304 x  32
    2 conv     64  3 x 3 / 1   304 x 304 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    3 max          2 x 2 / 2   304 x 304 x  64   ->   152 x 152 x  64
    4 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    5 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
    6 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    7 max          2 x 2 / 2   152 x 152 x 128   ->    76 x  76 x 128
    8 conv    256  3 x 3 / 1    76 x  76 x 128   ->    76 x  76 x 256  3.407 BFLOPs
    9 conv    128  1 x 1 / 1    76 x  76 x 256   ->    76 x  76 x 128  0.379 BFLOPs
   10 conv    256  3 x 3 / 1    76 x  76 x 128   ->    76 x  76 x 256  3.407 BFLOPs
   11 max          2 x 2 / 2    76 x  76 x 256   ->

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!nvidia-smi

Mon Apr 19 19:10:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
os.environ['PATH'] += ':/usr/local/cuda/bin'
!rm -fr darknet
!git clone https://github.com/AlexeyAB/darknet/

Cloning into 'darknet'...
remote: Enumerating objects: 14949, done.
remote: Total 14949 (delta 0), reused 0 (delta 0), pack-reused 14949
Receiving objects: 100% (14949/14949), 13.33 MiB | 24.20 MiB/s, done.
Resolving deltas: 100% (10160/10160), done.


In [ ]:
!apt install gcc-5 g++-5 -y
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc 
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  cpp-5 gcc-5-base libasan2 libgcc-5-dev libisl15 libmpx0 libstdc++-5-dev
Suggested packages:
  gcc-5-locales g++-5-multilib gcc-5-doc libstdc++6-5-dbg gcc-5-multilib
  libgcc1-dbg libgomp1-dbg libitm1-dbg libatomic1-dbg libasan2-dbg
  liblsan0-dbg libtsan0-dbg libubsan0-dbg libcilkrts5-dbg libmpx0-dbg
  libquadmath0-dbg libstdc++-5-doc
The following NEW packages will be installed:
  cpp-5 g++-5 gcc-5 gcc-5-base libasan2 libgcc-5-dev libisl15 libmpx0
  libstdc++-5-dev
0 upgraded, 9 newly installed, 0 to remove and 31 not upgraded.
Need to get 29.1 MB of archives.
After this operation, 100 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 gcc-5-base amd64 5.5.0-12ubuntu1 [17.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libisl15 amd64 0.18-4 [548 kB]
Get:3 http://arch

In [ ]:
!pwd

/content


In [ ]:
%cd darknet
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1424:14: warning: unused variable ‘buff’ [-Wunused-variable]
         char buff[100];
              ^
./src/image_opencv.cpp:1400:9: warning: unused variable ‘it_tb_res’ [-Wunused-variable]
     int it_tb

In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights
!chmod a+x ./darknet

--2021-04-19 19:12:21--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M   106MB/s    in 2.2s    

2021-04-19 19:12:23 (106 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [ ]:
!pwd

/content/darknet


In [ ]:
!apt install ffmpeg libopencv-dev libgtk-3-dev python-numpy python3-numpy libdc1394-22 libdc1394-22-dev libjpeg-dev libtiff5-dev libavcodec-dev libavformat-dev libswscale-dev libxine2-dev libgstreamer1.0-dev libgstreamer-plugins-base1.0-dev libv4l-dev libtbb-dev qtbase5-dev libfaac-dev libmp3lame-dev libopencore-amrnb-dev libopencore-amrwb-dev libtheora-dev libvorbis-dev libxvidcore-dev x264 v4l-utils unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libjpeg-dev is already the newest version (8c-2ubuntu8).
libjpeg-dev set to manually installed.
libvorbis-dev is already the newest version (1.3.5-4.2).
libvorbis-dev set to manually installed.
python-numpy is already the newest version (1:1.13.3-2ubuntu1).
python-numpy set to manually installed.
libdc1394-22 is already the newest version (2.2.5-1).
libdc1394-22 set to manually installed.
libdc1394-22-dev is already the newest version (2.2.5-1).
libdc1394-22-dev set to manually installed.
libtbb-dev is already the newest version (2017~U7-8).
libtbb-dev set to manually installed.
libtiff5-dev is already the newest version (4.0.9-5ubuntu0.4).
libtiff5-dev set to manually installed.
qtbase5-dev is already the newest version (5.9.5+dfsg-0ubuntu2.5).
qtbase5-dev set to manually installed.
unzip is already the newest version (6.0-21ubuntu1.1).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0

In [ ]:
from google.colab import files

In [ ]:
uploaded = files.upload()

Saving videoplayback.mp4 to videoplayback.mp4


In [ ]:
!pwd  # /content/darknet/yoltest.mp4

/content/darknet


In [ ]:
!./darknet detector demo cfg/coco.data cfg/yolov3.cfg yolov3.weights -dont_show videoplayback.mp4  -i  0 -out_filename yolov3output.avi -thresh 0.6

In [ ]:
!./darknet detector demo cfg/coco.data cfg/yolov3.cfg yolov3.weights -dont_show videoplayback.mp4

Streaming output truncated to the last 5000 lines.

FPS:39.4 	 AVG_FPS:39.4
Objects:


FPS:39.3 	 AVG_FPS:39.4
Objects:


FPS:39.3 	 AVG_FPS:39.4
Objects:


FPS:39.4 	 AVG_FPS:39.4
Objects:


FPS:39.4 	 AVG_FPS:39.4
Objects:


FPS:39.3 	 AVG_FPS:39.4
Objects:


FPS:39.5 	 AVG_FPS:39.4
Objects:


FPS:39.2 	 AVG_FPS:39.4
Objects:


FPS:39.1 	 AVG_FPS:39.4
Objects:


FPS:39.0 	 AVG_FPS:39.4
Objects:


FPS:39.2 	 AVG_FPS:39.4
Objects:


FPS:39.5 	 AVG_FPS:39.4
Objects:


FPS:39.4 	 AVG_FPS:39.4
Objects:


FPS:39.4 	 AVG_FPS:39.4
Objects:


FPS:39.4 	 AVG_FPS:39.4
Objects:


FPS:39.5 	 AVG_FPS:39.4
Objects:


FPS:39.6 	 AVG_FPS:39.4
Objects:


FPS:39.6 	 AVG_FPS:39.4
Objects:


FPS:39.5 	 AVG_FPS:39.4
Objects:


FPS:39.5 	 AVG_FPS:39.4
Objects:


FPS:39.5 	 AVG_FPS:39.4
Objects:


FPS:39.5 	 AVG_FPS:39.4
Objects:


FPS:39.5 	 AVG_FPS:39.4
Objects:


FPS:39.6 	 AVG_FPS:39.4
Objects:


FPS:39.5 	 AVG_FPS:39.4
Objects:


FPS:39.3 	 AVG_FPS:39.4
Objects:


FPS:39.4 	 AVG_FPS:39.4
Objects:


FPS

In [ ]:
from google.colab import files
files.download('yolov3output.avi')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>